<a href="https://colab.research.google.com/github/Mateus-Brandao/Estudo/blob/main/Word2Vec_Word_Embedding.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
import pandas as pd
import numpy as np
import spacy

In [ ]:
!unzip /content/drive/MyDrive/AulasAlura/Word2VecWordEmbedding/treino.zip
!unzip /content/drive/MyDrive/AulasAlura/Word2VecWordEmbedding/teste.zip

Archive:  /content/drive/MyDrive/AulasAlura/Word2VecWordEmbedding/treino.zip
  inflating: treino.csv              
Archive:  /content/drive/MyDrive/AulasAlura/Word2VecWordEmbedding/teste.zip
  inflating: teste.csv               


In [ ]:
dados_teste = pd.read_csv('teste.csv')
dados_treino = pd.read_csv('treino.csv')

In [ ]:
"""
!pip install -U pip setuptools wheel
!pip install -U spacy
!python -m spacy download pt_core_news_sm
"""

'\n!pip install -U pip setuptools wheel\n!pip install -U spacy\n!python -m spacy download pt_core_news_sm\n!python -m spacy download pt_core_news_lg\n'

In [ ]:
!python -m spacy download pt_core_news_sm

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
     |████████████████████████████████| 13.0 MB 27.3 MB/s 
✔ Download and installation successful
You can now load the package via spacy.load('pt_core_news_sm')


In [ ]:
nlp = spacy.load("pt_core_news_sm")

In [ ]:
nlp

In [ ]:
texto = 'Rio de Janeiro é uma cidade maravilhosa dddd112  331sdaas'
doc = nlp(texto)

In [ ]:
doc.ents

(Rio de Janeiro, dddd112  )

In [ ]:
doc[0].is_stop

False

In [ ]:
doc[1].is_stop

True

In [ ]:
dados_treino.head()

,title,text,date,category,subcategory,link
0,"Após polêmica, Marine Le Pen diz que abomina n...",A candidata da direita nacionalista à Presidên...,2017-04-28,mundo,NaN,http://www1.folha.uol.com.br/mundo/2017/04/187...
1,"Macron e Le Pen vão ao 2º turno na França, em ...",O centrista independente Emmanuel Macron e a d...,2017-04-23,mundo,NaN,http://www1.folha.uol.com.br/mundo/2017/04/187...
2,"Apesar de larga vitória nas legislativas, Macr...",As eleições legislativas deste domingo (19) na...,2017-06-19,mundo,NaN,http://www1.folha.uol.com.br/mundo/2017/06/189...
3,"Governo antecipa balanço, e Alckmin anuncia qu...",O número de ocorrências de homicídios dolosos ...,2015-07-24,cotidiano,NaN,http://www1.folha.uol.com.br/cotidiano/2015/07...
4,"Após queda em maio, a atividade econômica sobe...","A economia cresceu 0,25% no segundo trimestre,...",2017-08-17,mercado,NaN,http://www1.folha.uol.com.br/mercado/2017/08/1...


In [ ]:
textos_para_tratamento = (titulos.lower() for titulos in dados_treino['title'])

In [ ]:
textos_para_tratamento

<generator object <genexpr> at 0x7f968ee74350>

In [ ]:
def trata_textos(doc):
  tokens_validos = []
  for token in doc:
    e_valido = not token.is_stop and token.is_alpha
    if e_valido:
      tokens_validos.append(token.text)
  
  if len(tokens_validos) > 2:
    return ' '.join(tokens_validos)

In [ ]:
trata_textos(doc)

'Rio Janeiro cidade maravilhosa'

In [ ]:
from time import time

t0 = time()

textos_tratados = [trata_textos(doc) for doc in nlp.pipe(textos_para_tratamento, batch_size = 1000, n_process = -1)]

tf = time() - t0
print(tf/60)

3.28491294781367


In [ ]:
titulos_tratados = pd.DataFrame({'titulo':textos_tratados})

In [ ]:
from gensim.models import Word2Vec

w2v_modelo = Word2Vec(sg = 0, window = 2, size = 300, min_count = 5,
                      alpha = 0.03, min_alpha = 0.007)


In [ ]:
print(len(titulos_tratados))

titulos_tratados = titulos_tratados.dropna().drop_duplicates()

print(len(titulos_tratados))

90000
84466


In [ ]:
lista_lista_tokens = [titulo.split(' ') for titulo in titulos_tratados['titulo']]

In [ ]:
import logging 

logging.basicConfig(format='%(asctime)s : - %(message)s', level = logging.INFO)

w2v_modelo = Word2Vec(sg = 0, window = 2, size = 300, min_count = 5,
                      alpha = 0.03, min_alpha = 0.007)

w2v_modelo.build_vocab(lista_lista_tokens, progress_per = 5000)

2022-07-18 18:12:45,725 : - collecting all words and their counts
2022-07-18 18:12:45,731 : - PROGRESS: at sentence #0, processed 0 words, keeping 0 word types
2022-07-18 18:12:45,756 : - PROGRESS: at sentence #5000, processed 31930 words, keeping 10193 word types
2022-07-18 18:12:45,778 : - PROGRESS: at sentence #10000, processed 63848 words, keeping 14989 word types
2022-07-18 18:12:45,795 : - PROGRESS: at sentence #15000, processed 95753 words, keeping 18279 word types
2022-07-18 18:12:45,814 : - PROGRESS: at sentence #20000, processed 127689 words, keeping 21033 word types
2022-07-18 18:12:45,835 : - PROGRESS: at sentence #25000, processed 159589 words, keeping 23491 word types
2022-07-18 18:12:45,855 : - PROGRESS: at sentence #30000, processed 191554 words, keeping 25494 word types
2022-07-18 18:12:45,875 : - PROGRESS: at sentence #35000, processed 223412 words, keeping 27330 word types
2022-07-18 18:12:45,895 : - PROGRESS: at sentence #40000, processed 255282 words, keeping 29053

In [ ]:
dir(w2v_modelo)

['__class__',
 '__contains__',
 '__delattr__',
 '__dict__',
 '__dir__',
 '__doc__',
 '__eq__',
 '__format__',
 '__ge__',
 '__getattribute__',
 '__getitem__',
 '__gt__',
 '__hash__',
 '__init__',
 '__init_subclass__',
 '__le__',
 '__lt__',
 '__module__',
 '__ne__',
 '__new__',
 '__reduce__',
 '__reduce_ex__',
 '__repr__',
 '__setattr__',
 '__sizeof__',
 '__str__',
 '__subclasshook__',
 '__weakref__',
 '_adapt_by_suffix',
 '_check_input_data_sanity',
 '_check_training_sanity',
 '_clear_post_train',
 '_do_train_epoch',
 '_do_train_job',
 '_get_job_params',
 '_get_thread_working_mem',
 '_job_producer',
 '_load_specials',
 '_log_epoch_end',
 '_log_epoch_progress',
 '_log_progress',
 '_log_train_end',
 '_minimize_model',
 '_raw_word_count',
 '_save_specials',
 '_set_train_params',
 '_smart_save',
 '_train_epoch',
 '_train_epoch_corpusfile',
 '_update_job_params',
 '_worker_loop',
 '_worker_loop_corpusfile',
 'accuracy',
 'alpha',
 'batch_words',
 'build_vocab',
 'build_vocab_from_freq',
 'ca

In [ ]:
w2v_modelo.train(lista_lista_tokens, total_examples = w2v_modelo.corpus_count,
                 epochs = 30)

2022-07-18 18:28:03,600 : - training model with 3 workers on 12924 vocabulary and 300 features, using sg=0 hs=0 sample=0.001 negative=5 window=2
2022-07-18 18:28:04,762 : - EPOCH 1 - PROGRESS: at 37.08% examples, 160077 words/s, in_qsize 3, out_qsize 2
2022-07-18 18:28:05,871 : - EPOCH 1 - PROGRESS: at 75.91% examples, 165102 words/s, in_qsize 3, out_qsize 2
2022-07-18 18:28:06,298 : - worker thread finished; awaiting finish of 2 more threads
2022-07-18 18:28:06,304 : - worker thread finished; awaiting finish of 1 more threads
2022-07-18 18:28:06,364 : - worker thread finished; awaiting finish of 0 more threads
2022-07-18 18:28:06,372 : - EPOCH - 1 : training on 540242 raw words (486043 effective words) took 2.7s, 177719 effective words/s
2022-07-18 18:28:07,528 : - EPOCH 2 - PROGRESS: at 38.94% examples, 169289 words/s, in_qsize 4, out_qsize 1
2022-07-18 18:28:08,581 : - EPOCH 2 - PROGRESS: at 72.20% examples, 161750 words/s, in_qsize 5, out_qsize 0
2022-07-18 18:28:09,309 : - worker 

(14584268, 16207260)

In [ ]:
w2v_modelo.wv.most_similar('google')

2022-07-18 18:30:35,638 : - precomputing L2-norms of word weight vectors


[('apple', 0.6044677495956421),
 ('facebook', 0.5637120008468628),
 ('amazon', 0.5299876928329468),
 ('airbnb', 0.503768265247345),
 ('uber', 0.4953097701072693),
 ('yahoo', 0.48489999771118164),
 ('tesla', 0.4713868498802185),
 ('software', 0.4664422273635864),
 ('volkswagen', 0.46550387144088745),
 ('waze', 0.46312078833580017)]

In [ ]:
w2v_modelo.wv.most_similar('microsoft')

[('amazon', 0.5604841709136963),
 ('unilever', 0.549654483795166),
 ('sky', 0.5477105379104614),
 ('spotify', 0.5383620858192444),
 ('walmart', 0.531922459602356),
 ('linkedin', 0.5120449066162109),
 ('canais', 0.5102347135543823),
 ('lego', 0.5070550441741943),
 ('tesla', 0.5066229104995728),
 ('braskem', 0.5035922527313232)]

In [ ]:
w2v_modelo.wv.most_similar('barcelona')

[('barça', 0.5763355493545532),
 ('bayern', 0.574711263179779),
 ('madrid', 0.5668283700942993),
 ('psg', 0.5613793730735779),
 ('munique', 0.5544523000717163),
 ('lazio', 0.5514475107192993),
 ('juventus', 0.5487446784973145),
 ('atlético', 0.5408505201339722),
 ('chelsea', 0.536598265171051),
 ('united', 0.5313850045204163)]

In [ ]:
w2v_modelo.wv.most_similar('messi')

[('suárez', 0.5869654417037964),
 ('cristiano', 0.5732539892196655),
 ('benzema', 0.536141037940979),
 ('tevez', 0.5237647294998169),
 ('ronaldo', 0.5219373106956482),
 ('psg', 0.5043196082115173),
 ('cavani', 0.49729764461517334),
 ('neymar', 0.49600374698638916),
 ('barça', 0.4932226538658142),
 ('mourinho', 0.4827943444252014)]

In [ ]:
w2v_modelo.wv.most_similar('gm')

[('embraer', 0.6763542294502258),
 ('volks', 0.6649438142776489),
 ('braskem', 0.6597031354904175),
 ('honda', 0.651060163974762),
 ('chrysler', 0.647871732711792),
 ('toyota', 0.6208133697509766),
 ('csn', 0.5903852581977844),
 ('itaipu', 0.5757691264152527),
 ('volkswagen', 0.5755221247673035),
 ('metalúrgicos', 0.5752568244934082)]

In [ ]:
# Treinamento em skipgram

w2v_modelo_sg = Word2Vec(sg = 1, window = 5, size = 300, min_count = 5,
                      alpha = 0.03, min_alpha = 0.007)

w2v_modelo_sg.build_vocab(lista_lista_tokens, progress_per = 5000)

w2v_modelo_sg.train(lista_lista_tokens, total_examples = w2v_modelo_sg.corpus_count,
                 epochs = 30)

2022-07-18 18:36:41,244 : - collecting all words and their counts
2022-07-18 18:36:41,248 : - PROGRESS: at sentence #0, processed 0 words, keeping 0 word types
2022-07-18 18:36:41,271 : - PROGRESS: at sentence #5000, processed 31930 words, keeping 10193 word types
2022-07-18 18:36:41,297 : - PROGRESS: at sentence #10000, processed 63848 words, keeping 14989 word types
2022-07-18 18:36:41,325 : - PROGRESS: at sentence #15000, processed 95753 words, keeping 18279 word types
2022-07-18 18:36:41,366 : - PROGRESS: at sentence #20000, processed 127689 words, keeping 21033 word types
2022-07-18 18:36:41,402 : - PROGRESS: at sentence #25000, processed 159589 words, keeping 23491 word types
2022-07-18 18:36:41,469 : - PROGRESS: at sentence #30000, processed 191554 words, keeping 25494 word types
2022-07-18 18:36:41,522 : - PROGRESS: at sentence #35000, processed 223412 words, keeping 27330 word types
2022-07-18 18:36:41,556 : - PROGRESS: at sentence #40000, processed 255282 words, keeping 29053

(14584131, 16207260)

In [ ]:
w2v_modelo_sg.wv.most_similar('google')

2022-07-18 18:39:54,813 : - precomputing L2-norms of word weight vectors


[('reguladores', 0.43514037132263184),
 ('android', 0.4087178111076355),
 ('yahoo', 0.39338499307632446),
 ('apple', 0.3907841444015503),
 ('patentes', 0.3773953318595886),
 ('waze', 0.37691962718963623),
 ('anunciantes', 0.37423813343048096),
 ('buffett', 0.3738279342651367),
 ('verizon', 0.37024855613708496),
 ('concorda', 0.3633447289466858)]

In [ ]:
w2v_modelo_sg.wv.most_similar('microsoft')

[('linkedin', 0.538314938545227),
 ('software', 0.478837788105011),
 ('kraft', 0.47541582584381104),
 ('chips', 0.4728188216686249),
 ('fertilizantes', 0.47192099690437317),
 ('syngenta', 0.4683956503868103),
 ('silício', 0.457232266664505),
 ('verizon', 0.45216792821884155),
 ('casino', 0.45216476917266846),
 ('farmacêuticas', 0.4482840299606323)]

In [ ]:
w2v_modelo_sg.wv.most_similar('barcelona')

[('celta', 0.5635251998901367),
 ('espanyol', 0.5205758213996887),
 ('sevilla', 0.5068633556365967),
 ('athletic', 0.501343846321106),
 ('supercopa', 0.4850257337093353),
 ('villarreal', 0.4747573733329773),
 ('madrid', 0.4654512405395508),
 ('monaco', 0.4638095796108246),
 ('wolfsburg', 0.45099446177482605),
 ('bilbao', 0.4365606904029846)]

In [ ]:
w2v_modelo_sg.wv.most_similar('messi')

[('suárez', 0.5247154831886292),
 ('barça', 0.49950870871543884),
 ('cavani', 0.4800060987472534),
 ('benzema', 0.46022969484329224),
 ('neymar', 0.4595707058906555),
 ('mascherano', 0.45786550641059875),
 ('celta', 0.45686304569244385),
 ('chuteiras', 0.45665740966796875),
 ('athletic', 0.45067453384399414),
 ('camp', 0.45044469833374023)]

In [ ]:
w2v_modelo_sg.wv.most_similar('gm')

[('metalúrgicos', 0.5842221975326538),
 ('motors', 0.5402358770370483),
 ('honda', 0.5111962556838989),
 ('audi', 0.48762252926826477),
 ('airbags', 0.48758113384246826),
 ('toyota', 0.48290982842445374),
 ('bmw', 0.4782467484474182),
 ('dupont', 0.4752248227596283),
 ('montadora', 0.47462549805641174),
 ('airbag', 0.46664997935295105)]

In [ ]:
w2v_modelo.wv.save_word2vec_format('/content/drive/MyDrive/AulasAlura/Word2VecWordEmbedding/modelo_cbow.txt', binary = False)

2022-07-18 18:44:04,251 : - storing 12924x300 projection weights into /content/drive/MyDrive/AulasAlura/Word2VecWordEmbedding/modelo_cbow.txt


In [ ]:
w2v_modelo_sg.wv.save_word2vec_format('/content/drive/MyDrive/AulasAlura/Word2VecWordEmbedding/modelo_sg.txt', binary = False)

2022-07-18 18:44:33,761 : - storing 12924x300 projection weights into /content/drive/MyDrive/AulasAlura/Word2VecWordEmbedding/modelo_sg.txt
